In [ ]:
#!pip install rpy2==3.5.1
#!pip install -Iv rpy2==3.4.2


In [ ]:
#%load_ext rpy2.ipython

In [ ]:
#%%R
#install.packages('tableone')
#install.packages("keras")
#install.packages("tensorflow")
#install.packages("tidyverse")
#install.packages("googledrive")
#install.packages("readr")
#install.packages("pacman")
#install.packages("caret")

In [ ]:
%%R
#install.packages("pacman")

In [ ]:
%%R
#install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘future.apply’, ‘lava’, ‘prodlim’, ‘ipred’, ‘pROC’, ‘recipes’




In [1]:
#%%R

library(googledrive)
library(keras)
library(tensorflow)
library(tidyverse)
library(readr)
library(dplyr)
library(caret)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


The following object is masked from ‘package:tensorflow’:

    train




In [ ]:
#This is python environment
#from google.colab import files

#uploaded = files.upload()


KeyboardInterrupt: ignored

In [2]:
#%%R

train_personas <- read_csv("/content/train_personas.csv")
test_hogares<- read_csv("/content/test_hogares.csv")
test_personas<- read_csv("/content/test_personas.csv")
train_hogares<- read_csv("/content/train_hogares.csv")

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 543109 Columns: 135
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): id, Dominio, Depto
dbl (128): Orden, Clase, Estrato1, P6020, P6040, P6050, P6090, P6100, P6210,...
lgl   (4): Cclasnr5, Cclasnr6, Imdies, Iof1es

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 66168 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): id, Dominio, Depto
dbl (13): Clase, P5000, P5010, P5090, P5100, P5130, P5140, Nper, Npersug, Li...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 219644 Columns: 63
── Column specifica

In [3]:
#%%R
#-------collapsing data from individual-level to household-level data-----------
#TRAIN PERSONAS: Check the number of unique values in 'id' column
n_distinct(train_personas$id)
n_distinct(train_hogares$id)


[1] 164960

[1] 164960

In [4]:
#%%R

#TRAIN PERSONAS: Using the aggregate() function calculate aggregated household income using train_personas databases
train_household_sum <- aggregate(cbind(Ingtot) ~ id, data = train_personas, FUN = sum)
train_household_max <- aggregate(cbind(Orden) ~ id, data = train_personas, FUN = max)


In [5]:
#%%R
family_head_train <- train_personas %>%
  filter(Orden == 1) %>%
  select(id, P6020, P6040, P6210)


In [6]:
#%%R
#Bind together
train_household <- train_household_sum %>%
  left_join(train_household_max, by = c("id"))

train_household <- train_household %>%
  left_join(family_head_train, by = c("id"))



In [7]:
#%%R
#Divide Ingtotal household into ingtotal per person to compare to Lp and Li
train_household <- train_household %>% mutate(Ingtot = (Ingtot / Orden))



#TEST PERSONAS: Check the number of unique values in 'id' column
n_distinct(test_personas$id)
n_distinct(test_hogares$id)


[1] 66168

[1] 66168

In [8]:
#%%R
#TEST PERSONAS: Using the aggregate() function calculate aggregated household income using train_personas databases
test_household_max <- aggregate(cbind(Orden) ~ id, data = test_personas, FUN = max)

family_head_test <- test_personas %>%
  filter(Orden == 1) %>%
  select(id, P6020, P6040, P6210)


In [9]:
#%%R
#Bind together
test_household <- family_head_test %>%
  left_join(test_household_max, by = c("id"))

In [10]:

#%%R
#----------------------merging data---------------------------------------------
#train data
train <- train_hogares %>%
  left_join(train_household, by = c("id"))
#test data
test <- test_hogares %>%
  left_join(test_household, by = c("id"))


In [11]:
#%%R
names(train)

[1] "id"          "Clase"       "Dominio"     "P5000"       "P5010"      
 [6] "P5090"       "P5100"       "P5130"       "P5140"       "Nper"       
[11] "Npersug"     "Ingtotug"    "Ingtotugarr" "Ingpcug"     "Li"         
[16] "Lp"          "Pobre"       "Indigente"   "Npobres"     "Nindigentes"
[21] "Fex_c"       "Depto"       "Fex_dpto"    "Ingtot"      "Orden"      
[26] "P6020"       "P6040"       "P6210"

In [12]:
#%%R
#Keep only relevant variables---------------------------------------------------
train_n <- train[, c("id", "Clase", "Li", "Lp", "Pobre", "Indigente", "P6020", "P6040", "Ingtot", "Nper", "P6210", "Depto","P5130", "P5140", "P5090", "P5000", "P5010")]


In [13]:
#%%R
train_n <- train_n %>%
  mutate(P5130 = ifelse(is.na(P5130), P5140, P5130)) %>%
  select(-P5140)

In [14]:
#%%R
test_n <- test[, c("id", "Clase", "Li", "Lp", "P6020", "P6040","Nper", "P6210", "Depto","P5130", "P5140", "P5090", "P5000", "P5010")] #Pobre, Indigente, and Ingtot don't exist.


In [15]:
#%%R
test_n <- test_n %>%
  mutate(P5130 = ifelse(is.na(P5130), P5140, P5130)) %>%
  select(-P5140)

In [16]:
#%%R
#Create the missing variables in the test database with NA values in all rows of the data frame
test_n$Pobre <- NA
test_n$Indigente <- NA
test_n$Ingtot <- NA

In [17]:
#%%R
glimpse(test_n) #16 variables
glimpse(train_n) #16 variables

Rows: 66,168
Columns: 16
$ id        <chr> "2a7ddc2779480d7f19834953", "a0c2e751e582fd49d564f308", "572…
$ Clase     <dbl> 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, …
$ Li        <dbl> 121449.45, 121449.45, 100763.34, 122251.78, 123664.36, 11958…
$ Lp        <dbl> 275594.0, 275594.0, 170886.4, 294093.7, 293293.4, 283609.8, …
$ P6020     <dbl> 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, …
$ P6040     <dbl> 58, 41, 50, 60, 37, 67, 50, 50, 33, 33, 62, 58, 69, 42, 36, …
$ Nper      <dbl> 5, 6, 2, 5, 2, 3, 3, 3, 3, 4, 4, 6, 1, 7, 3, 4, 3, 2, 2, 2, …
$ P6210     <dbl> 6, 5, 1, 5, 6, 4, 5, 5, 6, 5, 3, 3, 5, 3, 5, 3, 6, 6, 6, 3, …
$ Depto     <chr> "47", "47", "17", "05", "05", "17", "20", "68", "68", "50", …
$ P5130     <dbl> 400000, 500000, 250000, 600000, 2000000, 550000, 350000, 650…
$ P5090     <dbl> 1, 4, 1, 1, 2, 1, 1, 1, 3, 3, 1, 3, 1, 3, 3, 4, 3, 1, 3, 4, …
$ P5000     <dbl> 4, 4, 3, 4, 5, 5, 4, 4, 4, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 6, …
$ P5010     <db

In [18]:
#%%R
#Replace missings with mode in Ingtot (17.57% of observations are missings)
colSums(is.na(train_n))/nrow(train_n)*100

Mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

train_n$Ingtot[is.na(train_n$Ingtot)] <- Mode(train_n$Ingtot)


id        Clase           Li           Lp        Pobre    Indigente 
0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 
       P6020        P6040       Ingtot         Nper        P6210        Depto 
0.0006062076 0.0006062076 0.0006062076 0.0000000000 0.0006062076 0.0000000000 
       P5130        P5090        P5000        P5010 
0.0000000000 0.0000000000 0.0000000000 0.0000000000

In [19]:
#%%R
##Bind both databases together -------------------------------------------------
#Generate new variable that identifies the sample
test_n<-test_n %>% mutate(sample="test")
train_n<-train_n %>% mutate(sample="train")

#Bind together both databases
total_table<-rbind(test_n,train_n)
table(total_table$sample) #test 66168 | train 164960



  test  train 
 66168 164960 

In [20]:
#%%R
total_table$P6020 <- ifelse(total_table$P6020 == 2, 0, total_table$P6020)
glimpse(total_table)


Rows: 231,128
Columns: 17
$ id        <chr> "2a7ddc2779480d7f19834953", "a0c2e751e582fd49d564f308", "572…
$ Clase     <dbl> 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, …
$ Li        <dbl> 121449.45, 121449.45, 100763.34, 122251.78, 123664.36, 11958…
$ Lp        <dbl> 275594.0, 275594.0, 170886.4, 294093.7, 293293.4, 283609.8, …
$ P6020     <dbl> 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, …
$ P6040     <dbl> 58, 41, 50, 60, 37, 67, 50, 50, 33, 33, 62, 58, 69, 42, 36, …
$ Nper      <dbl> 5, 6, 2, 5, 2, 3, 3, 3, 3, 4, 4, 6, 1, 7, 3, 4, 3, 2, 2, 2, …
$ P6210     <dbl> 6, 5, 1, 5, 6, 4, 5, 5, 6, 5, 3, 3, 5, 3, 5, 3, 6, 6, 6, 3, …
$ Depto     <chr> "47", "47", "17", "05", "05", "17", "20", "68", "68", "50", …
$ P5130     <dbl> 400000, 500000, 250000, 600000, 2000000, 550000, 350000, 650…
$ P5090     <dbl> 1, 4, 1, 1, 2, 1, 1, 1, 3, 3, 1, 3, 1, 3, 3, 4, 3, 1, 3, 4, …
$ P5000     <dbl> 4, 4, 3, 4, 5, 5, 4, 4, 4, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 6, …
$ P5010     <d

In [21]:
#%%R

#Change Clase from 2 to 0 resto
total_table$Clase <- ifelse(total_table$Clase == 2, 0, total_table$Clase)
glimpse(total_table)


Rows: 231,128
Columns: 17
$ id        <chr> "2a7ddc2779480d7f19834953", "a0c2e751e582fd49d564f308", "572…
$ Clase     <dbl> 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, …
$ Li        <dbl> 121449.45, 121449.45, 100763.34, 122251.78, 123664.36, 11958…
$ Lp        <dbl> 275594.0, 275594.0, 170886.4, 294093.7, 293293.4, 283609.8, …
$ P6020     <dbl> 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, …
$ P6040     <dbl> 58, 41, 50, 60, 37, 67, 50, 50, 33, 33, 62, 58, 69, 42, 36, …
$ Nper      <dbl> 5, 6, 2, 5, 2, 3, 3, 3, 3, 4, 4, 6, 1, 7, 3, 4, 3, 2, 2, 2, …
$ P6210     <dbl> 6, 5, 1, 5, 6, 4, 5, 5, 6, 5, 3, 3, 5, 3, 5, 3, 6, 6, 6, 3, …
$ Depto     <chr> "47", "47", "17", "05", "05", "17", "20", "68", "68", "50", …
$ P5130     <dbl> 400000, 500000, 250000, 600000, 2000000, 550000, 350000, 650…
$ P5090     <dbl> 1, 4, 1, 1, 2, 1, 1, 1, 3, 3, 1, 3, 1, 3, 3, 4, 3, 1, 3, 4, …
$ P5000     <dbl> 4, 4, 3, 4, 5, 5, 4, 4, 4, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 6, …
$ P5010     <d

In [22]:
#%%R
#Ajustar las variables para que sean factores
total_table <- total_table %>%  mutate(Clase = as.factor(Clase),
                                       P6020 = as.factor(P6020), #sexo 1: hombre 0: mujer
                                       Pobre = as.factor(Pobre), # Pobre=1 No pobre=0
                                       P5090 = as.factor(P5090), #a. Propia, totalmente pagada b. Propia, la están pagando c. En arriendo o subarriendo d. En usufructo e. Posesión sin titulo (ocupante f. Otra
                                       P6210 = as.factor(P6210), #a. Ninguno b. Preescolar c. Básica primaria (1o - 5o) d. Básica secundaria (6o - 9o) e. Media (10o - 13o) f. Superior o universitaria g. No sabe, no informa
                                       Depto = as.factor(Depto),
                                       P5000 = as.factor(P5000),
                                       P5010 = as.factor(P5010),
                                       Indigente = as.factor(Indigente)) # Indigente=1 No indigente=0

glimpse(total_table)


Rows: 231,128
Columns: 17
$ id        <chr> "2a7ddc2779480d7f19834953", "a0c2e751e582fd49d564f308", "572…
$ Clase     <fct> 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, …
$ Li        <dbl> 121449.45, 121449.45, 100763.34, 122251.78, 123664.36, 11958…
$ Lp        <dbl> 275594.0, 275594.0, 170886.4, 294093.7, 293293.4, 283609.8, …
$ P6020     <fct> 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, …
$ P6040     <dbl> 58, 41, 50, 60, 37, 67, 50, 50, 33, 33, 62, 58, 69, 42, 36, …
$ Nper      <dbl> 5, 6, 2, 5, 2, 3, 3, 3, 3, 4, 4, 6, 1, 7, 3, 4, 3, 2, 2, 2, …
$ P6210     <fct> 6, 5, 1, 5, 6, 4, 5, 5, 6, 5, 3, 3, 5, 3, 5, 3, 6, 6, 6, 3, …
$ Depto     <fct> 47, 47, 17, 05, 05, 17, 20, 68, 68, 50, 76, 08, 73, 66, 54, …
$ P5130     <dbl> 400000, 500000, 250000, 600000, 2000000, 550000, 350000, 650…
$ P5090     <fct> 1, 4, 1, 1, 2, 1, 1, 1, 3, 3, 1, 3, 1, 3, 3, 4, 3, 1, 3, 4, …
$ P5000     <fct> 4, 4, 3, 4, 5, 5, 4, 4, 4, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 6, …
$ P5010     <f

In [23]:
#%%R
total_table %>%
  filter(sample == "train") %>%
  glimpse()


#Check for imbalance in Pobre
prop.table(table(total_table$Pobre)) #79.98% de la muestra  no es pobre, 20% lo  es
#Desbalance no es extremo pues el 20% de la muestra es la clase minoritaria, por lo que se podría decir que hay un desbalance moderado


Rows: 164,960
Columns: 17
$ id        <chr> "8d0193e328305a042001a35f", "cf7491d7f2cc9c216bd009e7", "d90…
$ Clase     <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ Li        <dbl> 122809.5, 122809.5, 122809.5, 122809.5, 122809.5, 122809.5, …
$ Lp        <dbl> 289878.2, 289878.2, 289878.2, 289878.2, 289878.2, 289878.2, …
$ P6020     <fct> 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, …
$ P6040     <dbl> 41, 64, 38, 43, 67, 48, 48, 51, 48, 26, 21, 22, 61, 58, 50, …
$ Nper      <dbl> 5, 4, 3, 1, 5, 3, 1, 4, 4, 2, 1, 1, 3, 2, 3, 1, 3, 5, 2, 3, …
$ P6210     <fct> 5, 4, 6, 3, 3, 5, 6, 6, 6, 6, 6, 6, 4, 6, 5, 3, 4, 5, 4, 4, …
$ Depto     <fct> 05, 05, 05, 05, 05, 05, 05, 05, 05, 05, 05, 05, 05, 05, 05, …
$ P5130     <dbl> 400000, 250000, 700000, 450000, 500000, 400000, 700000, 7500…
$ P5090     <fct> 1, 3, 1, 1, 3, 1, 1, 1, 1, 3, 3, 3, 1, 1, 3, 4, 3, 1, 4, 1, …
$ P5000     <fct> 4, 5, 4, 3, 4, 3, 4, 4, 4, 4, 1, 1, 4, 3, 3, 3, 3, 4, 4, 4, …
$ P5010     <f


       0        1 
0.799806 0.200194 

In [24]:
#%%R
head(total_table)
table(total_table$sample) #test 66.168 | train 164.960
colnames(total_table) <- tolower(colnames(total_table))
names(total_table)

#Check for missing values
colSums(is.na(total_table))/nrow(total_table)*100

#Check the levels of each variable
variable_levels <- sapply(total_table, function(x) length(unique(x)))
variable_levels

#Vtable statistics
total_table_selected<- total_table %>% select(li, lp, p6020, p6040, pobre, indigente, ingtot, p5090, p6210, p5130, p5000, p5010)


id,Clase,Li,Lp,P6020,P6040,Nper,P6210,Depto,P5130,P5090,P5000,P5010,Pobre,Indigente,Ingtot,sample
<chr>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<chr>
2a7ddc2779480d7f19834953,1,121449.5,275594.0,1,58,5,6,47,400000,1,4,3,NA,NA,NA,test
a0c2e751e582fd49d564f308,1,121449.5,275594.0,1,41,6,5,47,500000,4,4,3,NA,NA,NA,test
57273d19e8464a5ff66a582b,0,100763.3,170886.4,1,50,2,1,17,250000,1,3,1,NA,NA,NA,test
418d052ff7878940ab938601,1,122251.8,294093.7,0,60,5,5,05,600000,1,4,3,NA,NA,NA,test
212a37fc17016a3c78f76852,1,123664.4,293293.4,1,37,2,6,05,2000000,2,5,2,NA,NA,NA,test
7b0b8c4814944383d6c8cef1,1,119588.8,283609.8,0,67,3,4,17,550000,1,5,3,NA,NA,NA,test



  test  train 
 66168 164960 

[1] "id"        "clase"     "li"        "lp"        "p6020"     "p6040"    
 [7] "nper"      "p6210"     "depto"     "p5130"     "p5090"     "p5000"    
[13] "p5010"     "pobre"     "indigente" "ingtot"    "sample"

id        clase           li           lp        p6020        p6040 
0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 4.326607e-04 4.326607e-04 
        nper        p6210        depto        p5130        p5090        p5000 
0.000000e+00 4.326607e-04 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 
       p5010        pobre    indigente       ingtot       sample 
0.000000e+00 2.862829e+01 2.862829e+01 2.862829e+01 0.000000e+00

id     clase        li        lp     p6020     p6040      nper     p6210 
   231128         2       300       300         3        96        23         8 
    depto     p5130     p5090     p5000     p5010     pobre indigente    ingtot 
       24       840         6        19        11         3         3     65200 
   sample 
        2

In [25]:
#%%R
##Convert to tibble to make it go faster
total_table <- as_tibble(total_table)

#Divide the total data to keep only the wanted training data variables (total income, age, sex)
train_data <- total_table  %>% filter(sample=="train")  %>% select(li, lp, p6020, p6040, p5090, nper, p6210, depto, p5130, p5000, p5010, pobre, indigente, ingtot)  %>% na.omit()

variables_numericas <- c("p5130", "li", "lp", "ingtot")
escalador <- preProcess(train_data[, variables_numericas],
                        method = c("center", "scale"))

train_data[, variables_numericas] <- predict(escalador, train_data[, variables_numericas])

train_data <- train_data  %>% mutate(p6020=factor(p6020,levels=c(0,1),labels=c("Woman","Man")))

# Calculate the percentage of zeros in the ingtot column
percentage_zeros <- 100 * mean(train_data$ingtot == 0, na.rm = TRUE)
print(percentage_zeros) # <1% are zeros. We will drop those observations

#Drop zeros in ingtot
train_data <- subset(train_data, ingtot != 0)
colSums(is.na(train_data))/nrow(train_data)*100

[1] 0


li        lp     p6020     p6040     p5090      nper     p6210     depto 
        0         0         0         0         0         0         0         0 
    p5130     p5000     p5010     pobre indigente    ingtot 
        0         0         0         0         0         0

In [26]:
#%%R

# Create a sequential neural network
model <- keras_model_sequential()

# Add layers to the network
model %>%
  layer_dense(units = 32, activation = "relu", input_shape = c(3)) %>%
  layer_dense(units = 16, activation = "relu") %>%
  layer_dense(units = 1, activation = "sigmoid")

# Compile the model
model %>% compile(
  loss = "binary_crossentropy",
  optimizer = optimizer_adam(),
  metrics = "accuracy"
)


In [ ]:
#%%R
# Assuming you have your data in a dataframe called 'data_df'
#predictors <- as.matrix(total_table[, c("p6020", "p6040", "p6210")])
#target <- as.numeric(total_table$pobre)  # Assuming poverty_status is a binary variable (0 or 1)





In [ ]:
#%%R
names(total_table)

 [1] "id"        "clase"     "li"        "lp"        "p6020"     "p6040"    
 [7] "nper"      "p6210"     "depto"     "p5130"     "p5090"     "p5000"    
[13] "p5010"     "pobre"     "indigente" "ingtot"    "sample"   


In [ ]:
#%%R
#x_train <- total_table  %>% filter(sample=="train")  %>% select(p6020, p6040, p6210)  %>% na.omit()
#y_train <- total_table  %>% filter(sample=="train")  %>% select(pobre)  %>% na.omit()
#x_test <- total_table  %>% filter(sample=="test")  %>% select(p6020, p6040, p6210)  %>% na.omit()
#y_test <- total_table  %>% filter(sample=="test")  %>% select(pobre)  %>% na.omit()




In [27]:
#%%R
# Filter and process the data
x_train <- total_table %>%
  filter(sample == "train") %>%
  select(p6020, p6040, p6210) %>%
  na.omit()

y_train <- total_table %>%
  filter(sample == "train") %>%
  pull(pobre) %>%
  na.omit()

x_test <- total_table %>%
  filter(sample == "test") %>%
  select(p6020, p6040, p6210) %>%
  na.omit()

y_test <- total_table %>%
  filter(sample == "test") %>%
  pull(pobre) %>%
  na.omit()

In [42]:
#%%R
# Convert factors to character strings in y_train and y_test
y_train <- as.character(y_train)
y_test <- as.character(y_test)

# Convert x_train and x_test to matrices
x_train <- as.matrix(x_train)
x_test <- as.matrix(x_test)

# Convert y_train and y_test to factors
y_train <- as.factor(y_train)
y_test <- as.factor(y_test)

In [34]:
#%%R
# Train the model
model %>% fit(
  x_train, y_train,
  epochs = 50,
  batch_size = 32,
  validation_data = list(x_test, y_test)
)

ERROR: ignored

In [33]:
#%%R
x_train <- x_train %>% mutate(p6020 = as.factor(p6020),
                              p6040 = as.vector(p6040),
                              p6210 = as.factor(p6210))

x_test <- x_test %>% mutate(p6020 = as.factor(p6020),
                              p6040 = as.vector(p6040),
                              p6210 = as.factor(p6210))



ERROR: ignored

In [ ]:
%%R
y_train <- y_train %>% mutate(pobre = as.factor(pobre))

y_test <- y_test %>% mutate(pobre = as.factor(pobre))


In [40]:
#%%R
# Train the model
model %>% fit(
  x_train, y_train,
  epochs = 50,
  batch_size = 32,
  validation_data = list(x_test, y_test)
)


ERROR: ignored